In [ ]:
#!pip install pandas tiktoken graphrag
#%pip install llama_index pyvis IPython
#%pip install --upgrade llama_index
#%pip install llama-index-readers-obsidian
#%pip install llama-index-llms-openai
#%pip install llama-index-llms-langchain
#%pip install llama-index-core llama-index-readers-file llama-index-llms-ollama llama-index-embeddings-huggingface

In [ ]:
#https://medium.com/@jeongiitae/from-rag-to-graphrag-what-is-the-graphrag-and-why-i-use-it-f75a7852c10c
#https://medium.com/@haiyangli_38602/make-knowledge-graph-rag-with-llamaindex-from-own-obsidian-notes-b20a350fa354

In [1]:
# Enabling Diagnostic Logging 
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [2]:
from llama_index.readers.obsidian import ObsidianReader
from llama_index.llms.openai import OpenAI
#from llama_index import LLMPredictor
from llama_index.core import ServiceContext
from llama_index.core.indices.knowledge_graph.base import KnowledgeGraphIndex
from llama_index.core.graph_stores.simple import SimpleGraphStore
from llama_index.core.storage import StorageContext
from IPython.display import Markdown, display

C:\Users\JohnDeHart\anaconda3\envs\graphrag\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
filepath = 'data'

In [4]:
use_context = {
 "temperature": 0,
 "model": "gpt-3.5-turbo",
 "chunk_size": 512
}

In [5]:
documents = ObsidianReader(filepath).load_data()
llm = OpenAI(temperature=use_context['temperature'], model=use_context['model'])
service_context = ServiceContext.from_defaults(llm=llm, 
 chunk_size=use_context['chunk_size'])

graph_store = SimpleGraphStore()
storage_context = StorageContext.from_defaults(graph_store=graph_store)

C:\Users\JohnDeHart\AppData\Local\Temp\ipykernel_31720\1681871332.py:3: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(llm=llm,


In [6]:
index = KnowledgeGraphIndex.from_documents(
 documents=documents,
 max_triplets_per_chunk=2,
 storage_context=storage_context,
 service_context=service_context,
 include_embeddings=True,
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK

In [7]:
query = "Tell me how the MQ-1 uses weapons"
query_engine = index.as_query_engine(
 include_text=True,
 response_mode="tree_summarize",
 embedding_mode="hybrid",
 similarity_top_k=5,
)

response = query_engine.query(query)
display(Markdown(f"<b>{response}</b>"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 25acf0bc-ff61-4120-a15e-472f4a1b8cc2: Scenario 1: Reconnaissance Mission

The Predator MQ-1 is deployed to gather i...
> Querying with idx: 25acf0bc-ff61-4120-a15e-472f4a1b8cc2: Scenario 1: Reconnaissance Mission

The Predator MQ-1 is deployed to gather i...
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 28a7c5c0-5365-4657-a3c3-b799e5d5b9ba: General Information

**Name:** Predator MQ-1

**Type:** Unmanned Aerial Vehic...
> Querying with idx: 28a7c5c0-5365-4657-a3c3-b799e5d5b9ba: General Information

**Name:** Predator MQ-1

**Type:** Unmanned Aerial Vehic

<b>The MQ-1 uses its laser designator to identify and designate targets, guiding weapons like the Hellfire missile to ensure precise engagement with minimal collateral damage.</b>

In [8]:
import codecs
from pyvis.network import Network

In [9]:
g = index.get_networkx_graph()
net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(g)

# Generate the HTML content
html_content = net.generate_html(name="example.html")

# Write the HTML content to a file with UTF-8 encoding
with codecs.open("example.html", "w", encoding="utf-8") as out_file:
    out_file.write(html_content)

In [72]:
storage_context.persist()